# **Estimation of water quality in Hong Kong using Sentinel-2 images in GEE and artificial neural network (ANN) in Google Colab environment**

This is the second part of python codes used in the article. The codes are tested inside Google Colab environment using Hong Kong water as the study area.

**Import required libraries & Initialize Google Earth Engine session**

In [2]:
import ee
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, root_mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
ee.Authenticate()
ee.Initialize()

# Step 1 - Match image data & in-situ station data

Sun glint correction & water mask are performed to each image in this step

In [ ]:
df_url = pd.read_csv("https://raw.githubusercontent.com/ivanhykwong/Marine-Water-Quality-Time-Series-HK/main/MarineQuality_2015-2020.csv")
df_url["Chlorophyll-a"].max()

In [7]:
# Load image data & in-situ station data
assetList = ee.data.getList({'id':"projects/ee-rsrohinsood/assets/S2_Py6S_mask_m"})
url = 'https://raw.githubusercontent.com/ivanhykwong/Marine-Water-Quality-Time-Series-HK/main/MarineQuality_2015-2020.csv'
station_list = ['TM2','TM3','TM4','TM5','TM6','TM7','TM8','SM1','SM2','SM3','SM4','SM5','SM6','SM7','SM9','SM10','SM11',
                'SM12','SM13','SM17','SM18','SM19','SM20','PM1','PM2','PM3','PM4','PM6','PM7','PM8','PM9','PM11','JM3',
                'JM4','DM1','DM2','DM3','DM4','DM5','NM1','NM2','NM3','NM5','NM6','NM8','MM1','MM2','MM3','MM4','MM5',
                'MM6','MM7','MM8','MM13','MM14','MM15','MM16','MM17','MM19','WM1','WM2','WM3','WM4','EM1','EM2','EM3',
                'VM1','VM2','VM4','VM5','VM6','VM7','VM8','VM12','VM14','VM15']
df_url = pd.read_csv(url)
df_url = df_url[df_url['Station'].isin(station_list)]
# print(assetList)
# print(len(assetList))
aoi = ee.Geometry.Polygon([[[113.800, 22.570],[113.800, 22.120],[114.514, 22.120],[114.514, 22.570]]])
df_data = pd.DataFrame()

pts = ee.FeatureCollection("projects/ee-rsrohinsood/assets/MonitoringStation_wgs84_76")
pt_list = pts.toList(pts.size())

for i in range(len(assetList)):
  # Extract image date
  assetid = assetList[i]['id']
  # print(assetid)
  d1 = ee.Image(assetid)
  d1_date = d1.date().format('yyyy-MM-dd')
  print(d1_date.getInfo())

  # sun glint correction by subtracting half of B11 from all bands
  # https://www.mdpi.com/2072-4292/1/4/697/htm
  # https://eatlas.org.au/data/uuid/2932dc63-9c9b-465f-80bf-09073aacaf1c
  swir_half = d1.select('B11').multiply(0.5)
  d1 = d1.subtract(swir_half)

  # water mask using MNDWI
  green = d1.select('B3')
  ndwi = d1.expression('(GREEN - SWIR) / (GREEN + SWIR)', {'GREEN': d1.select('B3'), 'SWIR': d1.select('B11')})
  mask = ndwi.gte(0.0).bitwiseAnd(green.gte(0.0)) # MNDWI >= 0
  d1 = d1.updateMask(mask)
  d1 = ee.Image(d1)

  # Find nearest date between image & station data
  df = df_url.copy()
  df['Dates'] = pd.to_datetime(df['Dates'], format='%Y-%m-%d')
  imagedate = datetime.strptime(d1_date.getInfo(), '%Y-%m-%d')
  df['Image_date'] = imagedate
  df['Date_compare'] = abs(df['Dates'] - imagedate)
  df = df.sort_values(by=['Date_compare'])
  df = df.drop_duplicates(subset=['Station'])

  if imagedate.year > 2020:
    continue

  df[['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']] = np.nan
  for pt in range(pt_list.length().getInfo()):
    pt1 = ee.Feature(pt_list.get(pt))
    pt1_buf = pt1.buffer(20)
    s2_dict = d1.reduceRegion(ee.Reducer.mean(), pt1_buf.geometry()).getInfo()
    n = pt1_buf.getInfo()['properties']['WaterStati']
    for b in ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']:
      df.loc[df['Station'] == n, b] = s2_dict[b]
  df = df.dropna(subset = ['B2'])
  df['n'] = df.shape[0]
  print(df.shape)
  
  df.head()

  # Combine all image dates
  df_data = pd.concat([df_data, df])

# Export tables
df_data.to_csv('df_data.csv')

2015-10-23
(54, 43)
2015-11-22
(62, 43)
2016-01-01
(73, 43)
2016-05-30
(31, 43)
2016-06-19
(56, 43)
2016-07-29
(42, 43)
2016-09-17
(71, 43)
2016-09-27
(70, 43)
2016-10-27
(46, 43)
2016-12-16
(74, 43)
2016-12-26
(59, 43)
2017-01-25
(76, 43)
2017-02-04
(74, 43)
2017-02-14
(72, 43)
2017-07-09
(44, 43)
2017-07-29
(22, 43)
2017-08-13
(35, 43)
2017-08-18
(45, 43)
2017-09-12
(48, 43)
2017-09-17
(70, 43)
2017-09-27
(68, 43)
2017-10-02
(43, 43)
2017-10-12
(69, 43)
2017-10-22
(76, 43)
2017-10-27
(75, 43)
2017-11-01
(76, 43)
2017-11-16
(55, 43)
2017-12-06
(76, 43)
2017-12-11
(76, 43)
2017-12-21
(71, 43)
2017-12-31
(76, 43)
2018-01-15
(74, 43)
2018-03-11
(49, 43)
2018-03-16
(72, 43)
2018-03-21
(34, 43)
2018-03-31
(33, 43)
2018-04-05
(22, 43)
2018-04-10
(30, 43)
2018-05-15
(22, 43)
2018-05-20
(39, 43)
2018-05-25
(17, 43)
2018-05-30
(50, 43)
2018-07-29
(48, 43)
2018-08-08
(20, 43)
2018-10-02
(68, 43)
2018-10-07
(68, 43)
2018-10-22
(41, 43)
2018-11-06
(73, 43)
2018-11-21
(72, 43)
2018-12-21
(56, 43)


KeyboardInterrupt: 

This block is written for inputing csv file as the data, skip this block if the dataframe is already loaded

In [ ]:
# for inputing csv file as the data, skip this block if the dataframe is already loaded
df_data = pd.read_csv('./data/df_data.csv')
df_data['Image_date'] = pd.to_datetime(df_data['Image_date'], format='%Y-%m-%d')
df_data['Date_compare'] = pd.to_timedelta(df_data['Date_compare'])
df_data = df_data.drop(columns=['Unnamed: 0'])

In [ ]:
df.shape()

# Step 2 - Extract observations & create variables

Extract observations with ≤1 day difference; remove outliers & compute band combinations

In [5]:
# Extract observations with ≤1 day difference

max_day_diff = 1

df = df_data[['Image_date', 'Dates', 'Date_compare', 'n',
              '5-day Biochemical Oxygen Demand mg_L', 'Ammonia Nitrogen mg_L', 'Chlorophyll-a ug_L', 'Dissolved Oxygen mg_L',
              'E. coli cfu_100mL', 'Faecal Coliforms cfu_100mL', 'Nitrate Nitrogen mg_L', 'Nitrite Nitrogen mg_L',
              'Orthophosphate Phosphorus mg_L', 'pH', 'Salinity psu', 'Secchi Disc Depth M', 'Silica mg_L',
              'Suspended Solids mg_L', 'Temperature C', 'Total Inorganic Nitrogen mg_L', 'Total Kjeldahl Nitrogen mg_L',
              'Total Nitrogen mg_L', 'Total Phosphorus mg_L', 'Turbidity NTU', 'Unionised Ammonia mg_L', 'Volatile Suspended Solids mg_L',
              'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A', 'B11', 'B12']].copy()

df = df.rename(columns={'Image_date': 'Image_Date', 'Dates': 'Station_Date',
                        '5-day Biochemical Oxygen Demand mg_L': 'BOD', 'Ammonia Nitrogen mg_L': 'AmNi', 'Chlorophyll-a ug_L': 'Chla', 'Dissolved Oxygen mg_L': 'DO',
                        'E. coli cfu_100mL': 'Ecoli', 'Faecal Coliforms cfu_100mL': 'FC', 'Nitrate Nitrogen mg_L': 'NitraNi', 'Nitrite Nitrogen mg_L': 'NitriNi',
                        'Orthophosphate Phosphorus mg_L': 'OrPh', 'pH': 'pH', 'Salinity psu': 'Sal', 'Secchi Disc Depth M': 'SDD', 'Silica mg_L': 'Si',
                        'Suspended Solids mg_L': 'SS', 'Temperature C': 'Temp', 'Total Inorganic Nitrogen mg_L': 'TIN', 'Total Kjeldahl Nitrogen mg_L': 'TKN',
                        'Total Nitrogen mg_L': 'ToNi', 'Total Phosphorus mg_L': 'ToPh', 'Turbidity NTU': 'Tur', 'Unionised Ammonia mg_L': 'UnAm', 'Volatile Suspended Solids mg_L': 'VSS'})

df['Date_compare'] = pd.to_numeric(df['Date_compare'].dt.days)
df.insert(0, "Date", df["Image_Date"])
df['Image_Year'] = pd.DatetimeIndex(df['Image_Date']).year

df = df[(df['Date_compare'] <= max_day_diff) & (df['n'] >= 10)].copy().drop(columns=['Image_Date', 'Station_Date', 'Date_compare', 'n'])

# Remove outlier using Tukey’s fences method

Q1 = df.quantile(q=0.25)
Q3 = df.quantile(q=0.75)
IQR = Q3 - Q1
df = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR)))[['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8A']].any(axis=1)]

# Replace 0 to min/2 (avoid inf errors during evaluation)

wq = ['BOD', 'AmNi', 'Chla', 'DO', 'Ecoli', 'FC', 'NitraNi', 'NitriNi', 'OrPh', 'pH', 'Sal', 'SDD', 'Si', 'SS', 'Temp', 'TIN', 'TKN', 'ToNi', 'ToPh', 'Tur', 'UnAm', 'VSS']
for a in wq:
  df[a]=df[a].replace(0, np.NaN)
  df[a]=df[a].replace(np.NaN,df[a].min()/2)
df["Chla"].max()

53.0

In [40]:
df.to_csv('data/df_data_graphing_filter.csv')

In [ ]:
# Create independent variables

bands = ['B' + str(b) for b in [*range(1,8),'8A',11,12]]
wl = [443,490,560,665,705,740,783,865,1610,2190]  #wavelength in nm

# Multiply 10
for i in bands:
  df[i] = df[i]*10

# Square and cubic
for i in bands:
  df[i+'_2'] = df[i]**2
  df[i+'_3'] = df[i]**3

# Two-band ratio
for i in bands:
  for j in bands:
    if (i != j) & (i < j):
      df['NR_'+i+j] = ((df[i] - df[j]) / (df[i] + df[j])).clip(lower=-1.0, upper=1.0)

# Three-band ratio
for i in range(0,10):
  for j in range(0,10):
    for k in range(0,10):
      if (j == i+1) & (k == j+1):
        df['TB_'+bands[i]+bands[j]+bands[k]] = (((1/df[bands[i]]) - (1/df[bands[j]])) * df[bands[k]]).clip(lower=-1.0, upper=1.0)

# Line height algorithm
for i in range(0,10):
  for j in range(0,10):
    for k in range(0,10):
      if (j == i+1) & (k == j+1):
        df['LH_'+bands[i]+bands[j]+bands[k]] = df[bands[j]] - df[bands[i]] - ((df[bands[k]] - df[bands[i]]) * ((wl[j]-wl[i])/(wl[k]-wl[i])))

df.to_csv('./data/df_data_filter.csv')

This block is written for inputing csv file as the data, skip this block if the dataframe is already loaded

In [4]:
# for inputing csv file as the data, skip this block if the dataframe is already loaded
df = pd.read_csv('./data/df_data_filter.csv')
df = df.drop(columns=['Unnamed: 0'])
wq = ['BOD', 'AmNi', 'Chla', 'DO', 'Ecoli', 'FC', 'NitraNi', 'NitriNi', 'OrPh', 'pH', 'Sal', 'SDD', 'Si', 'SS', 'Temp', 'TIN', 'TKN', 'ToNi', 'ToPh', 'Tur', 'UnAm', 'VSS']

# Step 3 - Train artificial neural network (ANN) models

Include selection of optimal variables through cross-validations; Based on GridSearchCV and MLPRegressor function in Scikit-learn

In [5]:
# Define train and test datasets

df_train = df[df['Image_Year'] <= 2019].drop(columns=['Image_Year']).copy()
df_test = df[df['Image_Year'] == 2020].drop(columns=['Image_Year']).copy()
X_train = df_train.drop(columns = wq)
X_test = df_test.drop(columns = wq)

std_scaler = StandardScaler()
std_scaler.set_output(transform='pandas')

# Transform the data (centering and scaling features)
df_train_scaled = std_scaler.fit_transform(df_train)
X_train_scaled = std_scaler.fit_transform(X_train)
X_test_scaled = std_scaler.fit_transform(X_test)

df_scaled = std_scaler.fit_transform(df)

In [68]:
# Define ANN function based on MLPRegressor
# https://scikit-learn.org/stable/modules/neural_networks_supervised.html#neural-networks-supervised

def ANN(df_train, X_train, wq_name, first, seed):
  print('seed:' + str(seed))
  print('var:' + str(first))
  print(wq_name)
  c = df_train.corr().copy()
  c = c[wq_name][22:113]
  c = abs(c).sort_values(ascending=False)[0:first]
  var = c.index.tolist()

  X_train2 = X_train[var]
  X_test2 = X_test[var]
  Y_train = df_train[wq_name]
  Y_test = df_test[wq_name]

  hidden_layer_sizes = [2,4,5,6,8,10,(2,2),(4,4),(5,5),(6,6),(8,8),(10,10)]
  tuned_parameters = {'hidden_layer_sizes': hidden_layer_sizes, 'alpha': 10.0 ** -np.arange(1, 7)}
  clf = GridSearchCV(MLPRegressor(random_state=seed,activation='logistic', solver='lbfgs', max_iter=50000, early_stopping=True),
                     param_grid=tuned_parameters, scoring='r2', verbose=1, cv=5)
  clf.fit(X_train2, Y_train)
  nvar = len(var)
  best_layer = clf.best_estimator_.hidden_layer_sizes
  best_alpha = clf.best_estimator_.alpha

  regr = MLPRegressor(random_state=seed, hidden_layer_sizes=best_layer, alpha=best_alpha, activation='logistic', 
                      solver='lbfgs', max_iter=50000, early_stopping=True).fit(X_train2, Y_train)

  r_squared = regr.score(X_train2, Y_train)
  adjusted_r_squared = 1 - (1-r_squared)*(len(Y_train)-1)/(len(Y_train)-X_train2.shape[1]-1)

  # Evaluate model
  Y_train_pred = regr.predict(X_train2)
  Y_train_pred[Y_train_pred<0]=0.0
  corr_model = np.corrcoef(Y_train, Y_train_pred)[0, 1]
  rmse = root_mean_squared_error(Y_train, Y_train_pred)
  mae = mean_absolute_error(Y_train, Y_train_pred)
  smape = np.mean(2*(np.abs(Y_train_pred - Y_train))/(np.abs(Y_train)+np.abs(Y_train_pred)))
  print(wq_name + ': ' + str(best_layer) + ', alpha: ' + str(best_alpha) + ', best_score: ' + str(clf.best_score_) + ', R2: ' + str(r_squared) + ', RMSE: ' + str(rmse))

  # Test model
  Y_test_pred = regr.predict(X_test2)
  Y_test_pred[Y_test_pred<0]=0.0
  r_squared_test = regr.score(X_test2, Y_test)
  corr_test = np.corrcoef(Y_test, Y_test_pred)[0, 1]
  rmse_test = mean_squared_error(Y_test, Y_test_pred, squared=False)
  mae_test = mean_absolute_error(Y_test, Y_test_pred)
  smape_test = np.mean(2*(np.abs(Y_test_pred - Y_test))/(np.abs(Y_test)+np.abs(Y_test_pred)))
  print(wq_name + ': r_squared_test: ' + str(r_squared_test) + ', rmse_test: ' + str(rmse_test) +  ', smape_test: ' + str(smape_test))

  ANN_df = pd.DataFrame({'WQ': [wq_name], 'nvar':[nvar], 'var':[var], 'random_state':[seed],
                         'best_layer':[best_layer], 'best_alpha':[best_alpha], 'best_score':[clf.best_score_],
                         'r2': [r_squared], 'adjusted_r2': [adjusted_r_squared], 'corr_model': [corr_model],
                         'rmse': [rmse], 'mae': [mae], 'smape': [smape],
                         'r2_test': [r_squared_test], 'corr_test': [corr_test],
                         'rmse_test': [rmse_test], 'mae_test': [mae_test], 'smape_test': [smape_test]})
  return(ANN_df)

In [ ]:
def ANN_TimeSeries(df, wq_name, first, seed, n_splits=5):
    
    print(wq_name, "Started")
    print("nvar", first)
    print("Seed", seed)
    
    # Define the TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    ANN_results = []
    
    wq = ['BOD', 'AmNi', 'Chla', 'DO', 'Ecoli', 'FC', 'NitraNi', 'NitriNi', 'OrPh', 'pH', 'Sal', 'SDD', 'Si', 'SS', 'Temp', 'TIN', 'TKN', 'ToNi', 'ToPh', 'Tur', 'UnAm', 'VSS']

    for train_index, test_index in tscv.split(df):
        df_train, df_test = df.iloc[train_index], df.iloc[test_index]
        X_train, X_test = df_train.drop(columns=wq), df_test.drop(columns=wq)
        Y_train, Y_test = df_train[wq_name], df_test[wq_name]

        # Fit scaler on training data and transform both training and test data
        scaler_X = StandardScaler()
        scaler_Y = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)
        Y_train_scaled = scaler_Y.fit_transform(Y_train.values.reshape(-1, 1)).flatten()
        Y_test_scaled = scaler_Y.transform(Y_test.values.reshape(-1, 1)).flatten()
        
        print("\n--------------------------------------")
        print(f'Training split: {train_index[0]} to {train_index[-1]}, Testing split: {test_index[0]} to {test_index[-1]}')
        
        # Correlation-based feature selection
        c = df_train.corr().copy()
        c = c[wq_name][22:113]
        c = abs(c).sort_values(ascending=False)[0:first]
        var = c.index.tolist()
        
        X_train2 = X_train_scaled[:, [X_train.columns.get_loc(col) for col in var]]
        X_test2 = X_test_scaled[:, [X_test.columns.get_loc(col) for col in var]]

        hidden_layer_sizes = [2, 4, 5, 6, 8, 10, (2, 2), (4, 4), (5, 5), (6, 6), (8, 8), (10, 10)]
        tuned_parameters = {'hidden_layer_sizes': hidden_layer_sizes, 'alpha': 10.0 ** -np.arange(1, 7)}
        
        clf = GridSearchCV(
            MLPRegressor(random_state=seed, activation='logistic', solver='sgd', max_iter=10000, early_stopping=True),
            param_grid=tuned_parameters, scoring='r2', verbose=1, cv=5
        )
        
        clf.fit(X_train2, Y_train_scaled)
        nvar = len(var)
        best_layer = clf.best_estimator_.hidden_layer_sizes
        best_alpha = clf.best_estimator_.alpha

        regr = MLPRegressor(
            random_state=seed, hidden_layer_sizes=best_layer, alpha=best_alpha, activation='logistic',
            solver='sgd', max_iter=10000, early_stopping=True
        ).fit(X_train2, Y_train_scaled)

        r_squared = regr.score(X_train2, Y_train_scaled)
        adjusted_r_squared = 1 - (1 - r_squared) * (len(Y_train) - 1) / (len(Y_train) - X_train2.shape[1] - 1)

        # Evaluate model on training data
        Y_train_pred_scaled = regr.predict(X_train2)
        Y_train_pred_scaled[Y_train_pred_scaled < 0] = 0.0

        # Inverse transform the predictions and actual values
        Y_train_pred = scaler_Y.inverse_transform(Y_train_pred_scaled.reshape(-1, 1)).flatten()
        Y_train_unscaled = scaler_Y.inverse_transform(Y_train_scaled.reshape(-1, 1)).flatten()
        
        corr_model = np.corrcoef(Y_train_unscaled, Y_train_pred)[0, 1]
        rmse = mean_squared_error(Y_train_unscaled, Y_train_pred, squared=False)
        mae = mean_absolute_error(Y_train_unscaled, Y_train_pred)
        smape = np.mean(2 * (np.abs(Y_train_pred - Y_train_unscaled)) / (np.abs(Y_train_unscaled) + np.abs(Y_train_pred)))
        print(wq_name + ': ' + str(best_layer) + ', alpha: ' + str(best_alpha) + ', best_score: ' + str(clf.best_score_) + ', R2: ' + str(r_squared) + ', RMSE: ' + str(rmse))

        # Test model on testing data
        Y_test_pred_scaled = regr.predict(X_test2)
        Y_test_pred_scaled[Y_test_pred_scaled < 0] = 0.0
        
        # Inverse transform the predictions and actual values
        Y_test_pred = scaler_Y.inverse_transform(Y_test_pred_scaled.reshape(-1, 1)).flatten()
        Y_test_unscaled = scaler_Y.inverse_transform(Y_test_scaled.reshape(-1, 1)).flatten()
        
        r_squared_test = regr.score(X_test2, Y_test_scaled)
        corr_test = np.corrcoef(Y_test_unscaled, Y_test_pred)[0, 1]
        rmse_test = mean_squared_error(Y_test_unscaled, Y_test_pred, squared=False)
        mae_test = mean_absolute_error(Y_test_unscaled, Y_test_pred)
        smape_test = np.mean(2 * (np.abs(Y_test_pred - Y_test_unscaled)) / (np.abs(Y_test_unscaled) + np.abs(Y_test_pred)))
        print(wq_name + ': r_squared_test: ' + str(r_squared_test) + ', rmse_test: ' + str(rmse_test) + ', smape_test: ' + str(smape_test))

        ANN_df = pd.DataFrame({
            'WQ': [wq_name], 'nvar': [nvar], 'var': [var], 'random_state': [seed],
            'best_layer': [best_layer], 'best_alpha': [best_alpha], 'best_score': [clf.best_score_],
            'r2': [r_squared], 'adjusted_r2': [adjusted_r_squared], 'corr_model': [corr_model],
            'rmse': [rmse], 'mae': [mae], 'smape': [smape],
            'r2_test': [r_squared_test], 'corr_test': [corr_test],
            'rmse_test': [rmse_test], 'mae_test': [mae_test], 'smape_test': [smape_test]
        })

        ANN_results.append(ANN_df)
        print("--------------------------------------\n")
        

    print(wq_name, "Finished")
    
    
    # Combine all results into a single DataFrame
    return pd.concat(ANN_results, ignore_index=True)

wq = ['Chla', 'SS', 'Tur']
ANN_result_list = [ANN_TimeSeries(df, wq_name=value, first=f, seed=1) for value in wq for f in range(4, 13)]
ANN_result = pd.concat(ANN_result_list)

ANN_result

ANN_result.to_csv("data/ANN_results.csv")

In [85]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Assuming 'df' is your pandas DataFrame containing the time series data

print("Starting ANN training...")

wq = ['Chla', 'SS', 'Tur']
n_splits = 5
seed = 1

ANN_results = []

for wq_name in wq:
    print(f"\n{wq_name} Started")
    
    for first in range(4, 13):
        print(f"nvar {first}")
        print(f"Seed {seed}")
        
        # Define the TimeSeriesSplit
        tscv = TimeSeriesSplit(n_splits=n_splits)

        for i, (train_index, test_index) in enumerate(tscv.split(df)):
            df_train, df_test = df.iloc[train_index], df.iloc[test_index]
            X_train, X_test = df_train.drop(columns=wq), df_test.drop(columns=wq)
            Y_train, Y_test = df_train[wq_name], df_test[wq_name]

            # Fit scaler on training data and transform both training and test data
            scaler_X = StandardScaler()
            scaler_Y = StandardScaler()
            X_train_scaled = scaler_X.fit_transform(X_train)
            X_test_scaled = scaler_X.transform(X_test)
            Y_train_scaled = scaler_Y.fit_transform(Y_train.values.reshape(-1, 1)).flatten()
            Y_test_scaled = scaler_Y.transform(Y_test.values.reshape(-1, 1)).flatten()

            print("\n--------------------------------------")
            print(f'Training split: {train_index[0]} to {train_index[-1]}, Testing split: {test_index[0]} to {test_index[-1]}')

            # Correlation-based feature selection
            c = df_train.corr().copy()
            c = c[wq_name][22:113]
            c = abs(c).sort_values(ascending=False)[0:first]
            var = c.index.tolist()

            X_train2 = X_train_scaled[:, [X_train.columns.get_loc(col) for col in var]]
            X_test2 = X_test_scaled[:, [X_test.columns.get_loc(col) for col in var]]

            hidden_layer_sizes = [2, 4, 5, 6, 8, 10, (2, 2), (4, 4), (5, 5), (6, 6), (8, 8), (10, 10)]
            tuned_parameters = {'hidden_layer_sizes': hidden_layer_sizes, 'alpha': 10.0 ** -np.arange(1, 7)}

            clf = GridSearchCV(
                MLPRegressor(random_state=seed, activation='logistic', solver='sgd', max_iter=10000, early_stopping=True),
                param_grid=tuned_parameters, scoring='r2', verbose=1, cv=5
            )

            clf.fit(X_train2, Y_train_scaled)
            nvar = len(var)
            best_layer = clf.best_estimator_.hidden_layer_sizes
            best_alpha = clf.best_estimator_.alpha

            regr = MLPRegressor(
                random_state=seed, hidden_layer_sizes=best_layer, alpha=best_alpha, activation='logistic',
                solver='sgd', max_iter=10000, early_stopping=True
            ).fit(X_train2, Y_train_scaled)

            r_squared = regr.score(X_train2, Y_train_scaled)
            adjusted_r_squared = 1 - (1 - r_squared) * (len(Y_train) - 1) / (len(Y_train) - X_train2.shape[1] - 1)

            # Evaluate model on training data
            Y_train_pred_scaled = regr.predict(X_train2)
            Y_train_pred_scaled[Y_train_pred_scaled < 0] = 0.0

            # Inverse transform the predictions and actual values
            Y_train_pred = scaler_Y.inverse_transform(Y_train_pred_scaled.reshape(-1, 1)).flatten()
            Y_train_unscaled = scaler_Y.inverse_transform(Y_train_scaled.reshape(-1, 1)).flatten()

            corr_model = np.corrcoef(Y_train_unscaled, Y_train_pred)[0, 1]
            rmse = mean_squared_error(Y_train_unscaled, Y_train_pred, squared=False)
            mae = mean_absolute_error(Y_train_unscaled, Y_train_pred)
            smape = np.mean(2 * (np.abs(Y_train_pred - Y_train_unscaled)) / (np.abs(Y_train_unscaled) + np.abs(Y_train_pred)))
            print(f'{wq_name}: {best_layer}, alpha: {best_alpha}, best_score: {clf.best_score_}, R2: {r_squared}, RMSE: {rmse}')

            # Test model on testing data
            Y_test_pred_scaled = regr.predict(X_test2)
            Y_test_pred_scaled[Y_test_pred_scaled < 0] = 0.0

            # Inverse transform the predictions and actual values
            Y_test_pred = scaler_Y.inverse_transform(Y_test_pred_scaled.reshape(-1, 1)).flatten()
            Y_test_unscaled = scaler_Y.inverse_transform(Y_test_scaled.reshape(-1, 1)).flatten()

            r_squared_test = regr.score(X_test2, Y_test_scaled)
            corr_test = np.corrcoef(Y_test_unscaled, Y_test_pred)[0, 1]
            rmse_test = mean_squared_error(Y_test_unscaled, Y_test_pred, squared=False)
            mae_test = mean_absolute_error(Y_test_unscaled, Y_test_pred)
            smape_test = np.mean(2 * (np.abs(Y_test_pred - Y_test_unscaled)) / (np.abs(Y_test_unscaled) + np.abs(Y_test_pred)))
            print(f'{wq_name}: r_squared_test: {r_squared_test}, rmse_test: {rmse_test}, smape_test: {smape_test}')

            # Store results in a DataFrame
            ANN_df = pd.DataFrame({
                'WQ': [wq_name], 'nvar': [nvar], 'var': [var], 'random_state': [seed],
                'best_layer': [best_layer], 'best_alpha': [best_alpha], 'best_score': [clf.best_score_],
                'r2': [r_squared], 'adjusted_r2': [adjusted_r_squared], 'corr_model': [corr_model],
                'rmse': [rmse], 'mae': [mae], 'smape': [smape],
                'r2_test': [r_squared_test], 'corr_test': [corr_test],
                'rmse_test': [rmse_test], 'mae_test': [mae_test], 'smape_test': [smape_test],
                'train_index': [train_index], 'test_index': [test_index],
                'Y_train': [Y_train], 'Y_train_pred': [Y_train_pred],
                'Y_test': [Y_test], 'Y_test_pred': [Y_test_pred]
            })

            ANN_results.append(ANN_df)
            print("--------------------------------------\n")

        print(f"{wq_name} Finished")

print("All ANN training finished.")

# Combine all results into a single DataFrame
ANN_result = pd.concat(ANN_results, ignore_index=True)

# Save results to CSV
ANN_result.to_csv("data/ANN_results.csv", index=False)

print("Results saved to data/ANN_results.csv")


Starting ANN training...

Chla Started
nvar 4
Seed 1

--------------------------------------
Training split: 0 to 58, Testing split: 59 to 117
Fitting 5 folds for each of 72 candidates, totalling 360 fits


/home/rohinsood/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/rohinsood/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/rohinsood/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/rohinsood/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: 

Chla: 5, alpha: 0.09999999999999999, best_score: -0.41863679546210336, R2: -0.11965933228141634, RMSE: 8.519626360572248
Chla: r_squared_test: -0.2522673810543601, rmse_test: 3.087444340204991, smape_test: 0.7924321212676583
--------------------------------------


--------------------------------------
Training split: 0 to 117, Testing split: 118 to 176
Fitting 5 folds for each of 72 candidates, totalling 360 fits


/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Chla: (5, 5), alpha: 1e-06, best_score: -1.5241188333767395, R2: 0.002995925518589626, RMSE: 6.401995568300206
Chla: r_squared_test: -0.00809488929252078, rmse_test: 5.209426741495381, smape_test: 0.9352248903800245
--------------------------------------


--------------------------------------
Training split: 0 to 176, Testing split: 177 to 235
Fitting 5 folds for each of 72 candidates, totalling 360 fits


...

Tur: (6, 6), alpha: 1e-06, best_score: -0.07337214263282073, R2: 0.007226597964838266, RMSE: 5.089177120083426
Tur: r_squared_test: -0.013707197568775475, rmse_test: 3.9441669689875587, smape_test: 0.6130988414955241
--------------------------------------


--------------------------------------
Training split: 0 to 235, Testing split: 236 to 294
Fitting 5 folds for each of 72 candidates, totalling 360 fits


/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Tur: 4, alpha: 0.09999999999999999, best_score: -0.009893301482409255, R2: 0.18772489362495592, RMSE: 4.484398721289528
Tur: r_squared_test: 0.12373960436759113, rmse_test: 5.0044147587281635, smape_test: 0.465429368343803
--------------------------------------


--------------------------------------
Training split: 0 to 294, Testing split: 295 to 353
Fitting 5 folds for each of 72 candidates, totalling 360 fits


/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Tur: 4, alpha: 1e-06, best_score: 0.019212969438120852, R2: 0.16105394799960582, RMSE: 4.668935024911024
Tur: r_squared_test: 0.2441555953569362, rmse_test: 1.8598073933149515, smape_test: 0.3394345495019868
--------------------------------------

Tur Finished
All ANN training finished.
Results saved to data/ANN_results.csv


In [ ]:
df = pd.read_csv("data/df_data_filter.csv")
df2 = pd.read_csv("data/df_data_graphing_filter.csv")

In [86]:
df = pd.read_csv("data/ANN_results.csv")

# Assuming df is your DataFrame
filtered_df = df[df['WQ'] == 'Chla']

lowest_rmse_entry_chla = filtered_df.loc[filtered_df['rmse_test'].idxmin()]

print('Lowest rmse_test value:', lowest_rmse_entry_chla['rmse_test'])
print('Other properties:', lowest_rmse_entry_chla)


Lowest rmse_test value: 2.405374546949454
Other properties: WQ                                                           Chla
nvar                                                            9
var             ['TB_B2B3B4', 'LH_B3B4B5', 'LH_B7B8AB11', 'B8A...
random_state                                                    1
best_layer                                                      5
best_alpha                                               0.000001
best_score                                              -0.156779
r2                                                       0.238763
adjusted_r2                                              0.208448
corr_model                                               0.622966
rmse                                                     4.861464
mae                                                      2.671804
smape                                                    0.844137
r2_test                                                  0.143191
corr_test       